In [924]:
import requests
import pandas as pd
import numpy as np
import re
import os
from bs4 import BeautifulSoup
from datetime import datetime
import time
import subprocess
from operator import itemgetter
from collections import OrderedDict
def ordered_union(*lists):
    return list(OrderedDict.fromkeys(item for lst in lists for item in lst))

In [1474]:
import sys
del sys.modules['utils']
from utils import excel_to_html, push_to_github

In [1916]:
class Ranking:
    

    def __init__(self, wta_dict, atp_dict, 
                 date = None, day = None,
                 wta_last_year_dict = None, atp_last_year_dict = None,
                 wta_this_week_dict = None, atp_this_week_dict = None,
                wta_url = None, atp_url = None, 
                 wta_referer = None, atp_referer = None,
                wta_score_cookie = None, wta_detail_cookie = None, 
                 atp_score_cookie = None, atp_detail_cookie = None):

        self.date = date
        self.day = day
        self.wta_dict = wta_dict
        self.atp_dict = atp_dict

        self.wta_this_week_dict = wta_this_week_dict
        self.atp_this_week_dict = atp_this_week_dict
        
        self.slams = ['澳网','法网','温网','美网']
        self.slams_dict = {x:2000 for x in self.slams}
        self.wta_forced = ['印第安维尔斯','迈阿密','马德里','罗马','蒙特利尔','辛辛那提','北京']
        self.wta_forced_dict = {x:1000 for x in self.wta_forced}
        self.wta_uncombined = ['多哈','迪拜','武汉']
        self.wta_uncombined_dict = {x:1000 for x in self.wta_uncombined}
        
        self.atp_forced = ['印第安维尔斯','迈阿密','马德里','罗马','多伦多','辛辛那提','上海','巴黎']
        self.atp_forced_dict = {x:1000 for x in self.atp_forced}
        self.atp_uncombined = ['蒙特卡洛']
        self.atp_uncombined_dict = {x:1000 for x in self.atp_uncombined}
        
        self.this_wta = [k for k, v in sorted(wta_dict.items(), key=itemgetter(1), reverse=True)]
        self.this_atp = [k for k, v in sorted(atp_dict.items(), key=itemgetter(1), reverse=True)]

        
        # self.wta_this_week = wta_this_week
        if wta_this_week_dict is not None:
            self.wta_this_week = list(wta_this_week_dict.keys())
        # self.atp_this_week = atp_this_week
        if atp_this_week_dict is not None:
            self.atp_this_week = list(atp_this_week_dict.keys())
        
        self.wta_last_week = ordered_union(self.slams,self.wta_forced,self.wta_uncombined,self.this_wta)
        self.atp_last_week = ordered_union(self.slams,self.atp_forced,self.atp_uncombined,self.this_atp)        
        if wta_this_week_dict is not None:
            self.wta_list = ordered_union(self.wta_this_week, self.wta_last_week)
        else:
            self.wta_list = self.wta_last_week
        if atp_this_week_dict is not None:
            self.atp_list = ordered_union(self.atp_this_week, self.atp_last_week)
        else:
            self.atp_list = self.atp_last_week
        self.wta_df = None
        self.atp_df = None
        self.wta_champion_df = None
        self.atp_champion_df = None
        self.wta_this_week_df = None
        self.atp_this_week_df = None
        self.wta_this_week_detail_df = None
        self.atp_this_week_detail_df = None
        self.wta_last_year_dict = wta_last_year_dict
        self.atp_last_year_dict = atp_last_year_dict
        self.wta_url = wta_url
        self.atp_url = atp_url
        self.wta_referer = wta_referer
        self.atp_referer = atp_referer
        self.wta_score_cookie = wta_score_cookie
        self.atp_score_cookie = atp_score_cookie
        self.wta_detail_cookie = wta_detail_cookie
        self.atp_detail_cookie = atp_detail_cookie

    def this_week_fetch(self, tennis_type):
        if tennis_type == 'wta':
            if self.wta_this_week_dict is None:
                return
            url = self.wta_url + '/score'
            referer = self.wta_referer + '/score'
            cookie = self.wta_score_cookie
        elif tennis_type == 'atp':
            if self.atp_this_week_dict is None:
                return
            url = self.atp_url + '/score'
            referer = self.atp_referer + '/score'
            cookie = self.atp_score_cookie
            
        headers = {
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Language': 'zh-CN,zh;q=0.9',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'Origin': 'https://www.live-tennis.cn',
            'Referer': referer,
            'Sec-Ch-Ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
            'Sec-Ch-Ua-Mobile': '?0',
            'Sec-Ch-Ua-Platform': '"macOS"',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
            'X-Csrf-Token': 'A8rCqRYBqEv2EzCljeXFCqwj4yUwJXbsps80keEF',
            'X-Requested-With': 'XMLHttpRequest',
            'cookie': cookie
        }
                
        all_rows = []
        start = 0
        length = 100
        
        while True:
            data = {
                "draw": "2",
                'start': str(start),
                'length': str(length),
                "order[0][column]": "0",
                "order[0][dir]": "asc",
                "search[value]": "",
                "search[regex]": "false",
                "device": "0",
                
                "columns[0][data]": "user_id",
                "columns[0][name]": "user_id",
                "columns[0][searchable]": "true",
                "columns[0][orderable]": "true",
                "columns[0][search][value]": "",
                "columns[0][search][regex]": "false",
            
                "columns[1][data]": "username",
                "columns[1][name]": "username",
                "columns[1][searchable]": "true",
                "columns[1][orderable]": "true",
                "columns[1][search][value]": "",
                "columns[1][search][regex]": "false",
            
                "columns[2][data]": "fill_status",
                "columns[2][name]": "fill_status",
                "columns[2][searchable]": "true",
                "columns[2][orderable]": "true",
                "columns[2][search][value]": "",
                "columns[2][search][regex]": "false",
            
                "columns[3][data]": "day",
                "columns[3][name]": "day",
                "columns[3][searchable]": "true",
                "columns[3][orderable]": "true",
                "columns[3][search][value]": "",
                "columns[3][search][regex]": "false",
            
                "columns[4][data]": "score",
                "columns[4][name]": "score",
                "columns[4][searchable]": "true",
                "columns[4][orderable]": "true",
                "columns[4][search][value]": "",
                "columns[4][search][regex]": "false",
            
                "columns[5][data]": "players",
                "columns[5][name]": "players",
                "columns[5][searchable]": "true",
                "columns[5][orderable]": "true",
                "columns[5][search][value]": "",
                "columns[5][search][regex]": "false"
            }
        
            resp = requests.post(url, headers=headers, data=data)
            res_json = resp.json()
            data_list = res_json.get('data', [])
        
            if not data_list:  # 没有数据，跳出循环
                break
        
            for item in data_list:
                all_rows.append({
                    '主键': item.get('user_id'),
                    '用户名': item.get('username'),
                    '状态': item.get('fill_status'),
                    '存活天数': item.get('day'),
                    '积分': item.get('score'),
                    '明细': item.get('players'),
                })
        
            start += length  # 翻页，开始下一个区间
        
        if tennis_type == 'wta':
            self.wta_this_week_df = pd.DataFrame(all_rows)
        elif tennis_type == 'atp':
            self.atp_this_week_df = pd.DataFrame(all_rows)

    def fetch(self, tennis_type):
        if tennis_type == 'wta':
            referer = 'https://www.live-tennis.cn/zh/survivor/rank/WS'
            url = 'https://www.live-tennis.cn/zh/survivor/rank/2'
            cookie = 'theme=light; FCNEC=%5B%5B%22AKsRol_O58u6br0IMlLemyq6rayDuJp42ib8zS7rytAn23-Gpr_hlHV05k1-IcSWc4R0rCb7uwzEcyZlsmuRwus8IJiolIhXSAcYx2uMLcEIc2GRyR-Uo3h1JnnQhQJUWnc86n0_rzGVdI9labvo5loRSfTnzS09Cw%3D%3D%22%5D%5D; rttype=local; lang=zh; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IlladWlEQlRGTjFkVm9qb2VROWpBQmc9PSIsInZhbHVlIjoiZDdKTVwvWlBrejV0WE8wM2hBZjN0OHBOYTVMTTNTSVFQNHFsTmdrTkZJRk5WeFQ1S3hFZVdaXC9UMStqRkNydlI3cTQzdGVTXC9NRlVFREpIa1NWcjBaN2lvbFJTN2RERkpycU9BNW1yQkRnRjlyZGJyUFBNS3NrQzNLcWt6SFI0WldZeWJYXC9WV2JuR3Q2WHhWdnhQTHU4R2NYdW9uSXFoVzZ2T2ZOTWpTMEtkcz0iLCJtYWMiOiJjOTIzNGQyZjczMTBhZTdhN2M5ZGFlMDdkZWZlZDhmM2UzYmRlMWNmMWIyMDcwZDY2NGY4NDU1NGEzNjYzZDc1In0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IkgyRjAxbTBiSWZrbTdFdjBvUTNpclE9PSIsInZhbHVlIjoiNVl4TVJHNnUzSkp2THYzYVBNZDExdlNYVm5ZZEZwd0ZiVHNNSFNGZFdqbTQvSE9yRXNSSlBvMmExWTRLQTJYelJqT2p3VWRoRGJyRmdONGFpaUx5WHpaOHd6WUE5aHRpUXlqbzFNeVhlSjd1WG1jWC9leWtPRkV5dU5oQzVidUFqd2JLZkZLWldLVU5qWkx2cWNkWGpQV3Z6THN5dGVvaUh1WHpUOCtPY2luZ1pyQm95bXViWXcrbWQ5YTRiYkw0NDd3MEU3bFJuRWsxZCtpNHdnQ3VKMDZlb21JWDZHci9hSmFqUzZ6bVJldz0iLCJtYWMiOiJkNjIxMjliZjg1NWYzNTZhNDJlYjE0NzNhYTQxN2M0YTI4Y2Q0ZDY2YjJhNzQ0MjNiY2ZiZTI2Y2EzZGQ0YWVkIiwidGFnIjoiIn0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6ImZIZG1lMU5xQTNqcW9Ja0VLTVBEcUE9PSIsInZhbHVlIjoibFl0VDJKWlNWZXk0TUlOcGdwb3FvZGFKVEczUUdndFhvNy9ldDFFb241T01uRERvdXlTZ3lxZnBpK2hRS04zNjRlM3A5ZHcwMXZHWndUOVpnY2FNdG14OU1LaXFpMFowc2hDQW9KbWp4ODhHdENQK2tuYXhZVlRnSDByWVhlSnZxdFlVeU9QL1YvenNrNUttcWFRVjRsczVnRVJwQzh5SUdHQXM1bURIRkhOanY1RDRSTHp0WGJIY0pxUTNoelczM0pPOVdDUlNhZVB1eVZMbXBXSjh4Vjd0WERBazI2T0RSQm05QTk3ei8xcz0iLCJtYWMiOiJjOWYyNDZkN2Y0NDc4MDJhMTMxYWMxYTY2M2FiOTM2NjE3ODA0MzNhYmEwYzVjYzQ4NDE1NTAwMTkyOTQ2OWY0IiwidGFnIjoiIn0%3D; cstype=list; def_sex=WS; rktype=4; Hm_ct_3b995bf0c6a621a743d0cf009eaf5c8a=17*1*%E8%B0%B7%E6%AD%8C!2444*1*35965!2445*1*CORICwrfDh8OMw5nDlsOPw4nDkMKEwr3DicOJw43Dkg%3D%3DTOP; _gid=GA1.2.1688611968.1750391674; Hm_lvt_3b995bf0c6a621a743d0cf009eaf5c8a=1748242565,1750391675; HMACCOUNT=78BC3E462729494F; _gat=1; Hm_lpvt_3b995bf0c6a621a743d0cf009eaf5c8a=1750392664; _ga_856GGDGVGH=GS2.2.s1750391676$o757$g1$t1750392664$j60$l0$h0; XSRF-TOKEN=eyJpdiI6ImNHblRwcklFT1hrQkppSGxqZzNLcUE9PSIsInZhbHVlIjoicmF2aTVSbWU0UmxZMVNtWXZCUUJNUElFVmVDY0V4RTMvWU14d3h3YjhyMGxKK1RlUXpMRWhWazFwa3kyT2I5RE9NaWwvbWN1ajlpejN2TFJHUG9XdFkwTmRpT3hTRGZobDRrTEtSMFBTWDR2bHJwZ3pRVUttRWpVenA5cFJDc3YiLCJtYWMiOiJiMzlmNjllMGJjMThlZTU2NjFiM2FmZTc1ZGUyMTIwMTc5NTUzNDA5NzlhNGQ0MzE3ZTEzMjI5MjBiMGRkNWJmIiwidGFnIjoiIn0%3D; live_tennis_session=eyJpdiI6IkZoakk2T3FKWCtGMTNvR1pMWWNSQnc9PSIsInZhbHVlIjoibldNaWczTEVJM2ticWRFVFVodk16OUdISGFKQXNRc3NUSUNSZ3FxUDNwamVabzlrMXhDcGVwWW1kTWswdFdKM3ZndStpY3Btcmd4TUVxSTM5OTdJQWc1a21Xb1R5R1NnTmFOK2tJQnJFQWZBaHcxZFJxbHZFcWpKNit3S3VvTU0iLCJtYWMiOiIxMWQ1N2FjZjM4ZTY5NmJlMmQ1NWExNTkyYmI4NWIzYzFmYzAyNjRkODU1ZjUzN2NkNzNiNDUyNzI4ODYwYWZjIiwidGFnIjoiIn0%3D; _ga_7GW8TTD6GW=GS2.1.s1750391673$o804$g1$t1750392703$j20$l0$h0; _ga=GA1.1.985475189.1708225794'
        elif tennis_type == 'atp':
            referer = 'https://www.live-tennis.cn/zh/survivor/rank/MS'
            url = 'https://www.live-tennis.cn/zh/survivor/rank/1'
            cookie = 'theme=light; FCNEC=%5B%5B%22AKsRol_O58u6br0IMlLemyq6rayDuJp42ib8zS7rytAn23-Gpr_hlHV05k1-IcSWc4R0rCb7uwzEcyZlsmuRwus8IJiolIhXSAcYx2uMLcEIc2GRyR-Uo3h1JnnQhQJUWnc86n0_rzGVdI9labvo5loRSfTnzS09Cw%3D%3D%22%5D%5D; rttype=local; lang=zh; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IlladWlEQlRGTjFkVm9qb2VROWpBQmc9PSIsInZhbHVlIjoiZDdKTVwvWlBrejV0WE8wM2hBZjN0OHBOYTVMTTNTSVFQNHFsTmdrTkZJRk5WeFQ1S3hFZVdaXC9UMStqRkNydlI3cTQzdGVTXC9NRlVFREpIa1NWcjBaN2lvbFJTN2RERkpycU9BNW1yQkRnRjlyZGJyUFBNS3NrQzNLcWt6SFI0WldZeWJYXC9WV2JuR3Q2WHhWdnhQTHU4R2NYdW9uSXFoVzZ2T2ZOTWpTMEtkcz0iLCJtYWMiOiJjOTIzNGQyZjczMTBhZTdhN2M5ZGFlMDdkZWZlZDhmM2UzYmRlMWNmMWIyMDcwZDY2NGY4NDU1NGEzNjYzZDc1In0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IkgyRjAxbTBiSWZrbTdFdjBvUTNpclE9PSIsInZhbHVlIjoiNVl4TVJHNnUzSkp2THYzYVBNZDExdlNYVm5ZZEZwd0ZiVHNNSFNGZFdqbTQvSE9yRXNSSlBvMmExWTRLQTJYelJqT2p3VWRoRGJyRmdONGFpaUx5WHpaOHd6WUE5aHRpUXlqbzFNeVhlSjd1WG1jWC9leWtPRkV5dU5oQzVidUFqd2JLZkZLWldLVU5qWkx2cWNkWGpQV3Z6THN5dGVvaUh1WHpUOCtPY2luZ1pyQm95bXViWXcrbWQ5YTRiYkw0NDd3MEU3bFJuRWsxZCtpNHdnQ3VKMDZlb21JWDZHci9hSmFqUzZ6bVJldz0iLCJtYWMiOiJkNjIxMjliZjg1NWYzNTZhNDJlYjE0NzNhYTQxN2M0YTI4Y2Q0ZDY2YjJhNzQ0MjNiY2ZiZTI2Y2EzZGQ0YWVkIiwidGFnIjoiIn0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6ImZIZG1lMU5xQTNqcW9Ja0VLTVBEcUE9PSIsInZhbHVlIjoibFl0VDJKWlNWZXk0TUlOcGdwb3FvZGFKVEczUUdndFhvNy9ldDFFb241T01uRERvdXlTZ3lxZnBpK2hRS04zNjRlM3A5ZHcwMXZHWndUOVpnY2FNdG14OU1LaXFpMFowc2hDQW9KbWp4ODhHdENQK2tuYXhZVlRnSDByWVhlSnZxdFlVeU9QL1YvenNrNUttcWFRVjRsczVnRVJwQzh5SUdHQXM1bURIRkhOanY1RDRSTHp0WGJIY0pxUTNoelczM0pPOVdDUlNhZVB1eVZMbXBXSjh4Vjd0WERBazI2T0RSQm05QTk3ei8xcz0iLCJtYWMiOiJjOWYyNDZkN2Y0NDc4MDJhMTMxYWMxYTY2M2FiOTM2NjE3ODA0MzNhYmEwYzVjYzQ4NDE1NTAwMTkyOTQ2OWY0IiwidGFnIjoiIn0%3D; cstype=list; def_sex=WS; rktype=4; Hm_ct_3b995bf0c6a621a743d0cf009eaf5c8a=17*1*%E8%B0%B7%E6%AD%8C!2444*1*35965!2445*1*CORICwrfDh8OMw5nDlsOPw4nDkMKEwr3DicOJw43Dkg%3D%3DTOP; _gid=GA1.2.1688611968.1750391674; Hm_lvt_3b995bf0c6a621a743d0cf009eaf5c8a=1748242565,1750391675; HMACCOUNT=78BC3E462729494F; _gat=1; Hm_lpvt_3b995bf0c6a621a743d0cf009eaf5c8a=1750392664; _ga_856GGDGVGH=GS2.2.s1750391676$o757$g1$t1750392664$j60$l0$h0; XSRF-TOKEN=eyJpdiI6ImNHblRwcklFT1hrQkppSGxqZzNLcUE9PSIsInZhbHVlIjoicmF2aTVSbWU0UmxZMVNtWXZCUUJNUElFVmVDY0V4RTMvWU14d3h3YjhyMGxKK1RlUXpMRWhWazFwa3kyT2I5RE9NaWwvbWN1ajlpejN2TFJHUG9XdFkwTmRpT3hTRGZobDRrTEtSMFBTWDR2bHJwZ3pRVUttRWpVenA5cFJDc3YiLCJtYWMiOiJiMzlmNjllMGJjMThlZTU2NjFiM2FmZTc1ZGUyMTIwMTc5NTUzNDA5NzlhNGQ0MzE3ZTEzMjI5MjBiMGRkNWJmIiwidGFnIjoiIn0%3D; live_tennis_session=eyJpdiI6IkZoakk2T3FKWCtGMTNvR1pMWWNSQnc9PSIsInZhbHVlIjoibldNaWczTEVJM2ticWRFVFVodk16OUdISGFKQXNRc3NUSUNSZ3FxUDNwamVabzlrMXhDcGVwWW1kTWswdFdKM3ZndStpY3Btcmd4TUVxSTM5OTdJQWc1a21Xb1R5R1NnTmFOK2tJQnJFQWZBaHcxZFJxbHZFcWpKNit3S3VvTU0iLCJtYWMiOiIxMWQ1N2FjZjM4ZTY5NmJlMmQ1NWExNTkyYmI4NWIzYzFmYzAyNjRkODU1ZjUzN2NkNzNiNDUyNzI4ODYwYWZjIiwidGFnIjoiIn0%3D; _ga_7GW8TTD6GW=GS2.1.s1750391673$o804$g1$t1750392703$j20$l0$h0; _ga=GA1.1.985475189.1708225794'
        headers = {
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Language': 'zh-CN,zh;q=0.9',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'Origin': 'https://www.live-tennis.cn',
            'Referer': referer,
            'Sec-Ch-Ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
            'Sec-Ch-Ua-Mobile': '?0',
            'Sec-Ch-Ua-Platform': '"macOS"',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
            'X-Csrf-Token': 'A8rCqRYBqEv2EzCljeXFCqwj4yUwJXbsps80keEF',
            'X-Requested-With': 'XMLHttpRequest',
            'Cookie': cookie,
        }
        
        all_rows = []
        start = 0
        length = 100
        
        while True:
            data = {
                'draw': '2',
                'start': str(start),
                'length': str(length),
                'search[value]': '',
                'search[regex]': 'false',
                'order[0][column]': '0',
                'order[0][dir]': 'asc',
                'device': '0',
                # 这里完整 columns 参数保持一致，略写示例：
                'columns[0][data]': 'user_id',
                'columns[0][name]': 'user_id',
                'columns[0][searchable]': 'true',
                'columns[0][orderable]': 'true',
                'columns[0][search][value]': '',
                'columns[0][search][regex]': 'false',
                # ... 其它 columns 同理填写 ...
                'columns[1][data]': 'username',
                'columns[1][name]': 'username',
                'columns[1][searchable]': 'true',
                'columns[1][orderable]': 'true',
                'columns[1][search][value]': '',
                'columns[1][search][regex]': 'false',
                # 继续到 columns[6]
                'columns[2][data]': 'rank',
                'columns[2][name]': 'rank',
                'columns[2][searchable]': 'true',
                'columns[2][orderable]': 'true',
                'columns[2][search][value]': '',
                'columns[2][search][regex]': 'false',
                'columns[3][data]': 'score',
                'columns[3][name]': 'score',
                'columns[3][searchable]': 'true',
                'columns[3][orderable]': 'true',
                'columns[3][search][value]': '',
                'columns[3][search][regex]': 'false',
                'columns[4][data]': 'title_num',
                'columns[4][name]': 'title_num',
                'columns[4][searchable]': 'true',
                'columns[4][orderable]': 'true',
                'columns[4][search][value]': '',
                'columns[4][search][regex]': 'false',
                'columns[5][data]': 'event_num',
                'columns[5][name]': 'event_num',
                'columns[5][searchable]': 'true',
                'columns[5][orderable]': 'true',
                'columns[5][search][value]': '',
                'columns[5][search][regex]': 'false',
                'columns[6][data]': 'details',
                'columns[6][name]': 'details',
                'columns[6][searchable]': 'true',
                'columns[6][orderable]': 'true',
                'columns[6][search][value]': '',
                'columns[6][search][regex]': 'false',
            }
        
            resp = requests.post(url, headers=headers, data=data)
            res_json = resp.json()
            data_list = res_json.get('data', [])
        
            if not data_list:  # 没有数据，跳出循环
                break
        
            for item in data_list:
                all_rows.append({
                    '主键': item.get('user_id'),
                    '用户名': item.get('username'),
                    '排名': item.get('rank'),
                    '得分': item.get('score'),
                    '冠军': item.get('title_num'),
                    '参赛': item.get('event_num'),
                    '明细': item.get('details'),
                })
        
            start += length  # 翻页，开始下一个区间
        if tennis_type == 'wta':
            self.wta_df = pd.DataFrame(all_rows)
        elif tennis_type == 'atp':
            self.atp_df = pd.DataFrame(all_rows)
        
        
    def fetch_choice(self, tennis_type):
        if tennis_type == 'wta':
            if self.wta_this_week_dict is None:
                return
            url = self.wta_url + '/detail'
            referer = self.wta_referer + '/detail'
            cookie = self.wta_detail_cookie
        elif tennis_type == 'atp':
            if self.atp_this_week_dict is None:
                return
            url = self.atp_url + '/detail'
            referer = self.atp_referer + '/detail'
            cookie = self.atp_detail_cookie
            
        headers = {
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Language': 'zh-CN,zh;q=0.9',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'Origin': 'https://www.live-tennis.cn',
            'Referer': referer,
            'Sec-Ch-Ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
            'Sec-Ch-Ua-Mobile': '?0',
            'Sec-Ch-Ua-Platform': '"macOS"',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
            'X-Csrf-Token': 'A8rCqRYBqEv2EzCljeXFCqwj4yUwJXbsps80keEF',
            'X-Requested-With': 'XMLHttpRequest',
            'cookie': cookie
        }
        
                
        all_rows = []
        start = 0
        length = 100
        
        while True:
            data = {
                'draw': '2',
                'start': str(start),
                'length': str(length),
                'search[value]': '',
                'search[regex]': 'false',
                'device': '0',
                'columns[0][data]': 'user_id',
                'columns[0][name]': 'user_id',
                'columns[0][searchable]': 'true',
                'columns[0][orderable]': 'true',
                'columns[0][search][value]': '',
                'columns[0][search][regex]': 'false',
                'columns[1][data]': 'username',
                'columns[1][name]': 'username',
                'columns[1][searchable]': 'true',
                'columns[1][orderable]': 'true',
                'columns[1][search][value]': '',
                'columns[1][search][regex]': 'false',
                'columns[2][data]': 'date_short',
                'columns[2][name]': 'date_short',
                'columns[2][searchable]': 'true',
                'columns[2][orderable]': 'true',
                'columns[2][search][value]': '',
                'columns[2][search][regex]': 'false',
                'columns[3][data]': 'player',
                'columns[3][name]': 'player',
                'columns[3][searchable]': 'true',
                'columns[3][orderable]': 'true',
                'columns[3][search][value]': '',
                'columns[3][search][regex]': 'false',
                'columns[4][data]': 'player_alt',
                'columns[4][name]': 'player_alt',
                'columns[4][searchable]': 'true',
                'columns[4][orderable]': 'true',
                'columns[4][search][value]': '',
                'columns[4][search][regex]': 'false',
                'columns[5][data]': 'fill_status',
                'columns[5][name]': 'fill_status',
                'columns[5][searchable]': 'true',
                'columns[5][orderable]': 'true',
                'columns[5][search][value]': '',
                'columns[5][search][regex]': 'false',
                'columns[6][data]': 'created_at',
                'columns[6][name]': 'created_at',
                'columns[6][searchable]': 'true',
                'columns[6][orderable]': 'true',
                'columns[6][search][value]': '',
                'columns[6][search][regex]': 'false',
                'order[0][column]': '0',
                'order[0][dir]': 'asc',
            }
        
            resp = requests.post(url, headers=headers, data=data)
            res_json = resp.json()
            data_list = res_json.get('data', [])
        
            if not data_list:  # 没有数据，跳出循环
                break
        
            for item in data_list:
                all_rows.append({
                    '主键': item.get('user_id'),
                    '用户名': item.get('username'),
                    '日期': item.get('date_short'),
                    '主选球员': item.get('player'),
                    '备选球员': item.get('player_alt'),
                    '状态': item.get('fill_status'),
                    '提交时间': item.get('created_at')
                })
        
            start += length  # 翻页，开始下一个区间
        
        if tennis_type == 'wta':
            self.wta_this_week_detail_df = pd.DataFrame(all_rows)
        elif tennis_type == 'atp':
            self.atp_this_week_detail_df = pd.DataFrame(all_rows)        
        

    def instant_ranking(self,tennis_type):
        if tennis_type == 'wta':
            df = self.wta_df
            last_year_match = list(self.wta_last_year_dict.keys())[0]
            this_week_match = list(self.wta_this_week_dict.keys())[0]
            if self.wta_this_week is not None:
                this_week_df = self.wta_this_week_df
        elif tennis_type == 'atp':
            df = self.atp_df
            last_year_match = list(self.atp_last_year_dict.keys())[0]
            this_week_match = list(self.atp_this_week_dict.keys())[0]
            if self.atp_this_week is not None:
                this_week_df = self.atp_this_week_df
                
        if this_week_df is not None:
            n = this_week_df.shape[0]
            rank_data = []
            for i in range(n):

                key = this_week_df.iloc[i]['主键']
                key_dict = {'主键': key}
    
                score = int(this_week_df.iloc[i]['积分'])
                if tennis_type == 'wta':
                    score_dict = {self.wta_this_week[0]: score}
                elif tennis_type == 'atp':
                    score_dict = {self.atp_this_week[0]: score}

                
                this_data = key_dict | score_dict
                rank_data.append(this_data)
            this_week_result = pd.DataFrame(rank_data)
            
        n = df.shape[0]
        rank_data = []
        for i in range(n):
            name = df.iloc[i]['用户名']

            # match = re.search(r'/([^/]+)\.\w+"', name)
            # user_type = match.group(1)
            
            pos = name.rfind('>')
            name = name[pos+1:] if pos != -1 else name
            name_dict  = {'用户名': name}

            key = df.iloc[i]['主键']
            key_dict = {'主键': key}
            
            summary = df.iloc[i]['明细']
            pattern = r'【([^（）()]+)\((\d+)\)】'
            matches = re.findall(pattern, summary)
            summary_dict = {city: int(num) for city, num in matches}
            this_data = name_dict | key_dict | summary_dict
            rank_data.append(this_data)
        result = pd.DataFrame(rank_data)
        result = result.drop(last_year_match, axis = 1)
        result = pd.merge(result, this_week_result, on = '主键', how = 'outer')
        result[this_week_match]
        
        

        
        
        
    
    def data_processing(self,tennis_type):
        this_week_df = None
        this_week_detail_df = None
        if tennis_type == 'wta':
            df = self.wta_df
            if self.wta_this_week is not None:
                this_week_df = self.wta_this_week_df
                this_week_detail_df = self.wta_this_week_detail_df
        elif tennis_type == 'atp':
            df = self.atp_df
            if self.atp_this_week is not None:
                this_week_df = self.atp_this_week_df
                this_week_detail_df = self.atp_this_week_detail_df

        if this_week_df is not None:
            n = this_week_df.shape[0]
            rank_data = []
            for i in range(n):
                name = this_week_df.iloc[i]['用户名']
    
                # match = re.search(r'/([^/]+)\.\w+"', name)
                # user_type = match.group(1)
                
                pos = name.rfind('>')
                name = name[pos+1:] if pos != -1 else name
                name_dict  = {'用户名': name}
                
                key = this_week_df.iloc[i]['主键']
                key_dict = {'主键': key}
    
                score = int(this_week_df.iloc[i]['积分'])
                if tennis_type == 'wta':
                    score_dict = {self.wta_this_week[0]: score}
                elif tennis_type == 'atp':
                    score_dict = {self.atp_this_week[0]: score}
    
                status = this_week_df.iloc[i]['状态']
                status_dict = {'状态': status}

                detail = this_week_df.iloc[i]['明细']
                detail_list = re.findall(r'【(.*?)】', detail) 
                detail_dict = {'明细': detail_list}
                
                this_data = key_dict | name_dict | status_dict | score_dict | detail_dict
                rank_data.append(this_data)
            this_week_result = pd.DataFrame(rank_data)

        if this_week_detail_df is not None:
            this_week_detail_df = this_week_detail_df[this_week_detail_df['日期'] == self.date]
            this_week_detail_df['用户名'] = this_week_detail_df['用户名'].apply(lambda x: x[x.rfind('>')+1:])
            this_week_detail_df = this_week_detail_df[['主键','用户名','主选球员','备选球员']]
        
        n = df.shape[0]
        rank_data = []
        for i in range(n):
            name = df.iloc[i]['用户名']

            # match = re.search(r'/([^/]+)\.\w+"', name)
            # user_type = match.group(1)
            
            pos = name.rfind('>')
            name = name[pos+1:] if pos != -1 else name
            name_dict  = {'用户名': name}

            key = df.iloc[i]['主键']
            key_dict = {'主键': key}            
            summary = df.iloc[i]['明细']
            pattern = r'【([^（）()]+)\((\d+)\)】'
            matches = re.findall(pattern, summary)
            summary_dict = {city: int(num) for city, num in matches}
            this_data = name_dict | key_dict | summary_dict
            rank_data.append(this_data)
            
        if tennis_type == 'wta':
            result = pd.DataFrame(rank_data, columns = ['用户名','主键'] + self.this_wta)
            result = result.reindex(columns=['用户名','主键'] + self.wta_last_week, fill_value=np.nan)
            result = result.fillna(0)
            result[self.wta_last_week] = result[self.wta_last_week].astype('Int64')
            
            
            result['上周总分'] = result.apply(lambda row: (
                row[self.slams].sum() 
                + row[self.wta_forced].astype('int').nlargest(6).sum()
                + row[self.wta_uncombined].astype('int').nlargest(1).sum()
                + row[row.index.difference(row[self.wta_forced].astype('int').nlargest(6).index.union(
                    row[self.wta_uncombined].astype('int').nlargest(1).index.union(
                        row[self.slams].index.union(
                            row[['用户名','主键']].index
                        ))))].astype('int').nlargest(7).sum()
            ), axis=1)

            this_week_match = list(self.wta_this_week_dict.keys())[0]
            if this_week_match in result.columns:
                result = result.drop(this_week_match, axis = 1)
            
            # result['上周排名'] = result['上周总分'].rank(ascending=False, method = 'min').astype('Int64')
            

            
                       
            if this_week_detail_df is not None:
                result = pd.merge(result, this_week_detail_df, on=['主键','用户名'],how='outer')

            if this_week_df is not None:
                if not this_week_df.empty:
                    result = pd.merge(result, this_week_result, on=['主键','用户名'],how='outer') 
                    result['状态'] = result['状态'].fillna('未参赛')
                else:
                    result['状态'] = '存活'
            else:
                result['状态'] = '本周完赛'
            

            if self.day > 1:
                result['备选球员'] = result.apply(lambda row: row['备选球员'] if row['状态'] == '存活' and row['主选球员'] not in row['明细']
                                              else '', axis=1)
                result['主选球员'] = result.apply(lambda row: row['主选球员'] if row['状态'] == '存活' and row['主选球员'] not in row['明细']
                                              else '', axis=1)                
                result = result.drop('明细', axis = 1)


            
            result = result.reindex(columns=['用户名','主键','状态','上周总分','上周排名','主选球员','备选球员'] + self.wta_list)
            # result = result.fillna(0)
            result[['主选球员','备选球员']] = result[['主选球员','备选球员']].fillna('')
            result['上周总分'] = result['上周总分'].fillna(0)
            result['上周排名'] = result['上周总分'].rank(ascending=False, method = 'min').astype('Int64')

            
            result[self.wta_list] = result[self.wta_list].fillna(0)
            result[self.wta_list] = result[self.wta_list].astype('Int64')
            result['总分'] = result.apply(lambda row: (
                row[self.slams].sum() 
                + row[self.wta_forced].astype('int').nlargest(6).sum()
                + row[self.wta_uncombined].astype('int').nlargest(1).sum()
                + row[row.index.difference(row[self.wta_forced].astype('int').nlargest(6).index.union(
                    row[self.wta_uncombined].astype('int').nlargest(1).index.union(
                        row[self.slams].index.union(
                            row[['用户名','主键','状态','上周总分','上周排名','主选球员','备选球员']].index
                        ))))].astype('int').nlargest(7).sum()
            ), axis=1)
            result['排名'] = result['总分'].rank(ascending=False, method = 'min').astype('Int64')
            result['升降'] = result['上周排名'] - result['排名']
            result = result.sort_values(by='排名')
            result = result[['排名','用户名','主键','总分','升降','状态','主选球员','备选球员']+self.wta_list]
            result.reset_index(drop = True, inplace = True)
            self.wta_champion_df = result



        
        elif tennis_type == 'atp':
            result = pd.DataFrame(rank_data, columns = ['用户名','主键'] + self.this_atp)
            result = result.reindex(columns=['用户名','主键'] + self.atp_last_week, fill_value=np.nan)
            result = result.fillna(0)
            result[self.atp_last_week] = result[self.atp_last_week].astype('Int64')
            
            
            result['上周总分'] = result.apply(lambda row: (
                row[self.slams].sum() 
                + row[self.atp_forced].astype('int').nlargest(5).sum()
                + row[row.index.difference(row[self.atp_forced].astype('int').nlargest(5).index.union(
                        row[self.slams].index.union(
                            row[['用户名','主键']].index
                        )))].astype('int').nlargest(10).sum()
            ), axis=1)

            this_week_match = list(self.atp_this_week_dict.keys())[0]
            if this_week_match in result.columns:
                result = result.drop(this_week_match, axis = 1)
            
            # result['上周排名'] = result['上周总分'].rank(ascending=False, method = 'min').astype('Int64')

            if this_week_detail_df is not None:
                result = pd.merge(result, this_week_detail_df, on=['主键','用户名'], how='outer')
            
            if this_week_df is not None:
                if not this_week_df.empty:
                    result = pd.merge(result, this_week_result, on=['主键','用户名'], how='outer') 
                    result['状态'] = result['状态'].fillna('未参赛')
                else:
                    result['状态'] = '存活'
            else:
                result['状态'] = '本周完赛'
            
        
            if self.day > 1:
                result['备选球员'] = result.apply(lambda row: row['备选球员'] if row['状态'] == '存活' and row['主选球员'] not in row['明细']
                                              else '', axis=1)
                result['主选球员'] = result.apply(lambda row: row['主选球员'] if row['状态'] == '存活' and row['主选球员'] not in row['明细']
                                              else '', axis=1)              
                result = result.drop('明细', axis = 1)
            
            result = result.reindex(columns=['用户名','主键','状态','上周总分','上周排名','主选球员','备选球员'] + self.atp_list)
            result['上周总分'] = result['上周总分'].fillna(0)
            result['上周排名'] = result['上周总分'].rank(ascending=False, method = 'min').astype('Int64')
            
            
            result[['主选球员','备选球员']] = result[['主选球员','备选球员']].fillna('')
            result[self.atp_list] = result[self.atp_list].fillna(0)
            result[self.atp_list] = result[self.atp_list].astype('Int64')
            result['总分'] = result.apply(lambda row: (
                row[self.slams].sum() 
                + row[self.atp_forced].astype('int').nlargest(5).sum()
                + row[row.index.difference(row[self.atp_forced].astype('int').nlargest(5).index.union(
                        row[self.slams].index.union(
                            row[['用户名','主键','状态','上周总分','上周排名','主选球员','备选球员']].index
                        )))].astype('int').nlargest(10).sum()
            ), axis=1)
            result['排名'] = result['总分'].rank(ascending=False, method = 'min').astype('Int64')
            result['升降'] = result['上周排名'] - result['排名']
            result = result.sort_values(by='排名')
            result = result[['排名','用户名','主键','总分','升降','状态','主选球员','备选球员']+self.atp_list]
            result.reset_index(drop = True, inplace = True)
            self.atp_champion_df = result

In [1923]:
date = '07/01'
day = 2
wta_dict = {'布里斯班':500,'阿德莱德':500,'澳网':2000,'新加坡':250,'阿布扎比':500,'多哈':1000,'迪拜':1000,'奥斯汀':250,
             '印第安维尔斯':1000,'迈阿密':1000,'查尔斯顿':500,'马德里':1000,'罗马':1000,'斯特拉斯堡':500,'法网':2000,'伦敦':500,'柏林':500,
           '巴特洪堡':500}
atp_dict = {'香港':250,'阿德莱德':250,'澳网':2000,'蒙彼利埃':250,'达拉斯':500,'马赛':250,'多哈':500,'迪拜':500,
             '印第安维尔斯':1000,'迈阿密':1000,'布加勒斯特':250,'蒙特卡洛':1000,'马德里':1000,'罗马':1000,'汉堡':500,'法网':2000,
           '马洛卡':250}

wta_this_week_dict = {'温网':2000}
wta_url = "https://www.live-tennis.cn/zh/survivor/event/91"
wta_referer = 'https://www.live-tennis.cn/zh/survivor/event/WC/2025/WS'
wta_score_cookie = 'theme=light; FCNEC=%5B%5B%22AKsRol_O58u6br0IMlLemyq6rayDuJp42ib8zS7rytAn23-Gpr_hlHV05k1-IcSWc4R0rCb7uwzEcyZlsmuRwus8IJiolIhXSAcYx2uMLcEIc2GRyR-Uo3h1JnnQhQJUWnc86n0_rzGVdI9labvo5loRSfTnzS09Cw%3D%3D%22%5D%5D; rttype=local; lang=zh; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IlladWlEQlRGTjFkVm9qb2VROWpBQmc9PSIsInZhbHVlIjoiZDdKTVwvWlBrejV0WE8wM2hBZjN0OHBOYTVMTTNTSVFQNHFsTmdrTkZJRk5WeFQ1S3hFZVdaXC9UMStqRkNydlI3cTQzdGVTXC9NRlVFREpIa1NWcjBaN2lvbFJTN2RERkpycU9BNW1yQkRnRjlyZGJyUFBNS3NrQzNLcWt6SFI0WldZeWJYXC9WV2JuR3Q2WHhWdnhQTHU4R2NYdW9uSXFoVzZ2T2ZOTWpTMEtkcz0iLCJtYWMiOiJjOTIzNGQyZjczMTBhZTdhN2M5ZGFlMDdkZWZlZDhmM2UzYmRlMWNmMWIyMDcwZDY2NGY4NDU1NGEzNjYzZDc1In0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IkgyRjAxbTBiSWZrbTdFdjBvUTNpclE9PSIsInZhbHVlIjoiNVl4TVJHNnUzSkp2THYzYVBNZDExdlNYVm5ZZEZwd0ZiVHNNSFNGZFdqbTQvSE9yRXNSSlBvMmExWTRLQTJYelJqT2p3VWRoRGJyRmdONGFpaUx5WHpaOHd6WUE5aHRpUXlqbzFNeVhlSjd1WG1jWC9leWtPRkV5dU5oQzVidUFqd2JLZkZLWldLVU5qWkx2cWNkWGpQV3Z6THN5dGVvaUh1WHpUOCtPY2luZ1pyQm95bXViWXcrbWQ5YTRiYkw0NDd3MEU3bFJuRWsxZCtpNHdnQ3VKMDZlb21JWDZHci9hSmFqUzZ6bVJldz0iLCJtYWMiOiJkNjIxMjliZjg1NWYzNTZhNDJlYjE0NzNhYTQxN2M0YTI4Y2Q0ZDY2YjJhNzQ0MjNiY2ZiZTI2Y2EzZGQ0YWVkIiwidGFnIjoiIn0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6ImZIZG1lMU5xQTNqcW9Ja0VLTVBEcUE9PSIsInZhbHVlIjoibFl0VDJKWlNWZXk0TUlOcGdwb3FvZGFKVEczUUdndFhvNy9ldDFFb241T01uRERvdXlTZ3lxZnBpK2hRS04zNjRlM3A5ZHcwMXZHWndUOVpnY2FNdG14OU1LaXFpMFowc2hDQW9KbWp4ODhHdENQK2tuYXhZVlRnSDByWVhlSnZxdFlVeU9QL1YvenNrNUttcWFRVjRsczVnRVJwQzh5SUdHQXM1bURIRkhOanY1RDRSTHp0WGJIY0pxUTNoelczM0pPOVdDUlNhZVB1eVZMbXBXSjh4Vjd0WERBazI2T0RSQm05QTk3ei8xcz0iLCJtYWMiOiJjOWYyNDZkN2Y0NDc4MDJhMTMxYWMxYTY2M2FiOTM2NjE3ODA0MzNhYmEwYzVjYzQ4NDE1NTAwMTkyOTQ2OWY0IiwidGFnIjoiIn0%3D; cstype=list; Hm_ct_3b995bf0c6a621a743d0cf009eaf5c8a=17*1*%E8%B0%B7%E6%AD%8C!2444*1*35965!2445*1*CORICwrfDh8OMw5nDlsOPw4nDkMKEwr3DicOJw43Dkg%3D%3DTOP; HMACCOUNT=78BC3E462729494F; Hm_lvt_3b995bf0c6a621a743d0cf009eaf5c8a=1750391675; _gid=GA1.2.1515404107.1751086118; Hm_lpvt_3b995bf0c6a621a743d0cf009eaf5c8a=1751125834; _ga_856GGDGVGH=GS2.2.s1751125073$o771$g1$t1751125834$j2$l0$h0; XSRF-TOKEN=eyJpdiI6Ikk2VGZxb2hDR1RSSmVwZkI3YU9oMHc9PSIsInZhbHVlIjoiYkxSU3NCK3NSNkZpdlY2VlV3NmNVT2t2Skc0NFUrWnd6ejVsb2Rwb0hlU2xDTU5Jb0RBYlpGczJMR1VyY1huWEtlNEF2VHh5MjhoN21meHMrbkdZTEV6dHNONWtNYjZLYjFBWFZZMUpvbzZXeFlpU0FSZVhJdEI0VFQySHRnbksiLCJtYWMiOiIwYTJjZjU3M2MwOTNjNTY0YjJiN2ZjMWE1ODVhMTJhMTFhMGNlNzdlMjJiNTRmNzI2MGRiYjk0Njg3NzI0N2NlIiwidGFnIjoiIn0%3D; live_tennis_session=eyJpdiI6IllueFFpR3c2YlM0d0ZmT283Q3lDY1E9PSIsInZhbHVlIjoiMGVtVHBJS1l5SzZaaUtxWG5uejA2ckozcFRGRkRvemgzajR6dVFoYkVyaXZLUkh2TjZTY0REMmpyTnNMbGNHbnhhK3E5Q3ZTNHlwQVhHQmNSRTVzbHZtdE00aFJnNUVhK1lCWkFNTnFDRnZUaTBvcEFiQnh0cGFEOW9PM1prMHkiLCJtYWMiOiJhYzU0ZmI4ZTk2YmJiNmM5MjU1ZmU3MzI5Mzc3NTM5MDkxN2RlMWNlMjFiMzFlYjcyYzVlOTBjZDljZTI1ZGZjIiwidGFnIjoiIn0%3D; _gat=1; _ga_7GW8TTD6GW=GS2.1.s1751125060$o818$g1$t1751125956$j57$l0$h0; _ga=GA1.1.985475189.1708225794'
wta_detail_cookie = 'theme=light; FCNEC=%5B%5B%22AKsRol_O58u6br0IMlLemyq6rayDuJp42ib8zS7rytAn23-Gpr_hlHV05k1-IcSWc4R0rCb7uwzEcyZlsmuRwus8IJiolIhXSAcYx2uMLcEIc2GRyR-Uo3h1JnnQhQJUWnc86n0_rzGVdI9labvo5loRSfTnzS09Cw%3D%3D%22%5D%5D; rttype=local; lang=zh; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IlladWlEQlRGTjFkVm9qb2VROWpBQmc9PSIsInZhbHVlIjoiZDdKTVwvWlBrejV0WE8wM2hBZjN0OHBOYTVMTTNTSVFQNHFsTmdrTkZJRk5WeFQ1S3hFZVdaXC9UMStqRkNydlI3cTQzdGVTXC9NRlVFREpIa1NWcjBaN2lvbFJTN2RERkpycU9BNW1yQkRnRjlyZGJyUFBNS3NrQzNLcWt6SFI0WldZeWJYXC9WV2JuR3Q2WHhWdnhQTHU4R2NYdW9uSXFoVzZ2T2ZOTWpTMEtkcz0iLCJtYWMiOiJjOTIzNGQyZjczMTBhZTdhN2M5ZGFlMDdkZWZlZDhmM2UzYmRlMWNmMWIyMDcwZDY2NGY4NDU1NGEzNjYzZDc1In0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IkgyRjAxbTBiSWZrbTdFdjBvUTNpclE9PSIsInZhbHVlIjoiNVl4TVJHNnUzSkp2THYzYVBNZDExdlNYVm5ZZEZwd0ZiVHNNSFNGZFdqbTQvSE9yRXNSSlBvMmExWTRLQTJYelJqT2p3VWRoRGJyRmdONGFpaUx5WHpaOHd6WUE5aHRpUXlqbzFNeVhlSjd1WG1jWC9leWtPRkV5dU5oQzVidUFqd2JLZkZLWldLVU5qWkx2cWNkWGpQV3Z6THN5dGVvaUh1WHpUOCtPY2luZ1pyQm95bXViWXcrbWQ5YTRiYkw0NDd3MEU3bFJuRWsxZCtpNHdnQ3VKMDZlb21JWDZHci9hSmFqUzZ6bVJldz0iLCJtYWMiOiJkNjIxMjliZjg1NWYzNTZhNDJlYjE0NzNhYTQxN2M0YTI4Y2Q0ZDY2YjJhNzQ0MjNiY2ZiZTI2Y2EzZGQ0YWVkIiwidGFnIjoiIn0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6ImZIZG1lMU5xQTNqcW9Ja0VLTVBEcUE9PSIsInZhbHVlIjoibFl0VDJKWlNWZXk0TUlOcGdwb3FvZGFKVEczUUdndFhvNy9ldDFFb241T01uRERvdXlTZ3lxZnBpK2hRS04zNjRlM3A5ZHcwMXZHWndUOVpnY2FNdG14OU1LaXFpMFowc2hDQW9KbWp4ODhHdENQK2tuYXhZVlRnSDByWVhlSnZxdFlVeU9QL1YvenNrNUttcWFRVjRsczVnRVJwQzh5SUdHQXM1bURIRkhOanY1RDRSTHp0WGJIY0pxUTNoelczM0pPOVdDUlNhZVB1eVZMbXBXSjh4Vjd0WERBazI2T0RSQm05QTk3ei8xcz0iLCJtYWMiOiJjOWYyNDZkN2Y0NDc4MDJhMTMxYWMxYTY2M2FiOTM2NjE3ODA0MzNhYmEwYzVjYzQ4NDE1NTAwMTkyOTQ2OWY0IiwidGFnIjoiIn0%3D; cstype=list; Hm_ct_3b995bf0c6a621a743d0cf009eaf5c8a=17*1*%E8%B0%B7%E6%AD%8C!2444*1*35965!2445*1*CORICwrfDh8OMw5nDlsOPw4nDkMKEwr3DicOJw43Dkg%3D%3DTOP; HMACCOUNT=78BC3E462729494F; Hm_lvt_3b995bf0c6a621a743d0cf009eaf5c8a=1750391675; _gid=GA1.2.1515404107.1751086118; _gat=1; Hm_lpvt_3b995bf0c6a621a743d0cf009eaf5c8a=1751125775; _ga_856GGDGVGH=GS2.2.s1751125073$o771$g1$t1751125776$j60$l0$h0; XSRF-TOKEN=eyJpdiI6InV0Um1NRksyNHRDMzZicWVMMlJleVE9PSIsInZhbHVlIjoiMkFhZTEzeEVMQ2R6dnNCblRwZnBTWEFydXBDNDRvd3hOdjJrZ1pXRWNWTjNlcWhzMUUyM015RHJKaXpTaXdRUSs5cnZHcldFa1RSd04zTW1qN3p6dG5XV0U3VnBlVEVWOEJpTGkvQlRZYTQzY2wwTkRuMGpTVUdWNFBLdFVzQTAiLCJtYWMiOiI2MDQ0MTQ0YmJlYjY2OGI0YWVhMTRhMjcyNmE2MDNjNzdjZThkYjUwZTQyMTg4NzU1OThhNzMyNWUzNjk5ODM2IiwidGFnIjoiIn0%3D; live_tennis_session=eyJpdiI6IitLODdka2xFUmZnZkNRdHMyN09nU0E9PSIsInZhbHVlIjoiRS9VcjRiMTJqTmVDeENRTEluQzZ4K3RzVmFzR2I2T1BjNGtqME9HNTJ0anVwRFUvY0R1ZGw1TWJKU0lYV0xqcHFvN0RwYmMrOEh6MU4zSUJmTXZEMlY3UDdYSW1hY0RmNFUwRHVOTWoyQnYxa1UrSktQckZnUVFCcWNiajk5ZzkiLCJtYWMiOiJhM2ViMTk1YzgzNDJlNmZlYWE3NGNhYTI4NmZiNWE5ODdhMmZlZDUxMWUzZWNjMjI4NzU1NzEyYzEzN2ZmYTljIiwidGFnIjoiIn0%3D; _ga_7GW8TTD6GW=GS2.1.s1751125060$o818$g1$t1751125834$j56$l0$h0; _ga=GA1.1.985475189.1708225794'


atp_this_week_dict = {'温网':2000}
atp_url = "https://www.live-tennis.cn/zh/survivor/event/92"
atp_referer = 'https://www.live-tennis.cn/zh/survivor/event/WC/2025/MS'
atp_score_cookie = 'theme=light; FCNEC=%5B%5B%22AKsRol_O58u6br0IMlLemyq6rayDuJp42ib8zS7rytAn23-Gpr_hlHV05k1-IcSWc4R0rCb7uwzEcyZlsmuRwus8IJiolIhXSAcYx2uMLcEIc2GRyR-Uo3h1JnnQhQJUWnc86n0_rzGVdI9labvo5loRSfTnzS09Cw%3D%3D%22%5D%5D; rttype=local; lang=zh; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IlladWlEQlRGTjFkVm9qb2VROWpBQmc9PSIsInZhbHVlIjoiZDdKTVwvWlBrejV0WE8wM2hBZjN0OHBOYTVMTTNTSVFQNHFsTmdrTkZJRk5WeFQ1S3hFZVdaXC9UMStqRkNydlI3cTQzdGVTXC9NRlVFREpIa1NWcjBaN2lvbFJTN2RERkpycU9BNW1yQkRnRjlyZGJyUFBNS3NrQzNLcWt6SFI0WldZeWJYXC9WV2JuR3Q2WHhWdnhQTHU4R2NYdW9uSXFoVzZ2T2ZOTWpTMEtkcz0iLCJtYWMiOiJjOTIzNGQyZjczMTBhZTdhN2M5ZGFlMDdkZWZlZDhmM2UzYmRlMWNmMWIyMDcwZDY2NGY4NDU1NGEzNjYzZDc1In0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IkgyRjAxbTBiSWZrbTdFdjBvUTNpclE9PSIsInZhbHVlIjoiNVl4TVJHNnUzSkp2THYzYVBNZDExdlNYVm5ZZEZwd0ZiVHNNSFNGZFdqbTQvSE9yRXNSSlBvMmExWTRLQTJYelJqT2p3VWRoRGJyRmdONGFpaUx5WHpaOHd6WUE5aHRpUXlqbzFNeVhlSjd1WG1jWC9leWtPRkV5dU5oQzVidUFqd2JLZkZLWldLVU5qWkx2cWNkWGpQV3Z6THN5dGVvaUh1WHpUOCtPY2luZ1pyQm95bXViWXcrbWQ5YTRiYkw0NDd3MEU3bFJuRWsxZCtpNHdnQ3VKMDZlb21JWDZHci9hSmFqUzZ6bVJldz0iLCJtYWMiOiJkNjIxMjliZjg1NWYzNTZhNDJlYjE0NzNhYTQxN2M0YTI4Y2Q0ZDY2YjJhNzQ0MjNiY2ZiZTI2Y2EzZGQ0YWVkIiwidGFnIjoiIn0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6ImZIZG1lMU5xQTNqcW9Ja0VLTVBEcUE9PSIsInZhbHVlIjoibFl0VDJKWlNWZXk0TUlOcGdwb3FvZGFKVEczUUdndFhvNy9ldDFFb241T01uRERvdXlTZ3lxZnBpK2hRS04zNjRlM3A5ZHcwMXZHWndUOVpnY2FNdG14OU1LaXFpMFowc2hDQW9KbWp4ODhHdENQK2tuYXhZVlRnSDByWVhlSnZxdFlVeU9QL1YvenNrNUttcWFRVjRsczVnRVJwQzh5SUdHQXM1bURIRkhOanY1RDRSTHp0WGJIY0pxUTNoelczM0pPOVdDUlNhZVB1eVZMbXBXSjh4Vjd0WERBazI2T0RSQm05QTk3ei8xcz0iLCJtYWMiOiJjOWYyNDZkN2Y0NDc4MDJhMTMxYWMxYTY2M2FiOTM2NjE3ODA0MzNhYmEwYzVjYzQ4NDE1NTAwMTkyOTQ2OWY0IiwidGFnIjoiIn0%3D; cstype=list; Hm_ct_3b995bf0c6a621a743d0cf009eaf5c8a=17*1*%E8%B0%B7%E6%AD%8C!2444*1*35965!2445*1*CORICwrfDh8OMw5nDlsOPw4nDkMKEwr3DicOJw43Dkg%3D%3DTOP; HMACCOUNT=78BC3E462729494F; Hm_lvt_3b995bf0c6a621a743d0cf009eaf5c8a=1750391675; _gid=GA1.2.1515404107.1751086118; _gat=1; Hm_lpvt_3b995bf0c6a621a743d0cf009eaf5c8a=1751127074; _ga_856GGDGVGH=GS2.2.s1751125073$o771$g1$t1751127074$j60$l0$h0; XSRF-TOKEN=eyJpdiI6IlBtMXhwZUpzT1owaFN0am5ycWVMZXc9PSIsInZhbHVlIjoiV0dyNURhUnA5QnpjdHhKaytPNE5BcjZSSFo0dnNIaDZDVEI5bUZJaG54cVdNMU5JRWJkd05aT2NQMFhoL2hZOEh3YUwzUVlxSC9XVHg1Z3dnaVJodVg0UTdWc1FPbElrSnNEQlhWbklTbFVJTFZJbU9xMFhYWGRvcHg4dzJIdFUiLCJtYWMiOiJkNjc0YjU5ZmY4ZjljZmNmYThjZTU3NmFiZTk2ZmM3ZGE4YmI0YTljZTczNzlhMTIzZDM2ODUzZmRkODI5MjE0IiwidGFnIjoiIn0%3D; live_tennis_session=eyJpdiI6Imd0UFRTcy8rMFd5SlZIWTYrVjYralE9PSIsInZhbHVlIjoianJ5Q2d6dEZLSjFVVE5aL21uMEhscVhkOXVudXZpWnA4YmpVZStwMzd0UWlxKzV3RnlqVzBTY2xNT2hJTkRSOXM5L2FWTnc0WjI3Vi9PenJjUUpuVFU1S3JXQXhROGNHMVJHcHdjeWNYdnpSbnNlVWlDVTRlL3FqMHozYVNoZ3IiLCJtYWMiOiI0NzVjNDcyNTFlMGZkMDY5ODQ2MjM2Y2I2NjA5ZmFlMmFjMTlhYjhjOTg1ZTBjYTI5OTA0MWQ0NjI0NDA2NjkzIiwidGFnIjoiIn0%3D; _ga_7GW8TTD6GW=GS2.1.s1751125060$o818$g1$t1751127082$j39$l0$h0; _ga=GA1.1.985475189.1708225794'
atp_detail_cookie = 'theme=light; FCNEC=%5B%5B%22AKsRol_O58u6br0IMlLemyq6rayDuJp42ib8zS7rytAn23-Gpr_hlHV05k1-IcSWc4R0rCb7uwzEcyZlsmuRwus8IJiolIhXSAcYx2uMLcEIc2GRyR-Uo3h1JnnQhQJUWnc86n0_rzGVdI9labvo5loRSfTnzS09Cw%3D%3D%22%5D%5D; rttype=local; lang=zh; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IlladWlEQlRGTjFkVm9qb2VROWpBQmc9PSIsInZhbHVlIjoiZDdKTVwvWlBrejV0WE8wM2hBZjN0OHBOYTVMTTNTSVFQNHFsTmdrTkZJRk5WeFQ1S3hFZVdaXC9UMStqRkNydlI3cTQzdGVTXC9NRlVFREpIa1NWcjBaN2lvbFJTN2RERkpycU9BNW1yQkRnRjlyZGJyUFBNS3NrQzNLcWt6SFI0WldZeWJYXC9WV2JuR3Q2WHhWdnhQTHU4R2NYdW9uSXFoVzZ2T2ZOTWpTMEtkcz0iLCJtYWMiOiJjOTIzNGQyZjczMTBhZTdhN2M5ZGFlMDdkZWZlZDhmM2UzYmRlMWNmMWIyMDcwZDY2NGY4NDU1NGEzNjYzZDc1In0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IkgyRjAxbTBiSWZrbTdFdjBvUTNpclE9PSIsInZhbHVlIjoiNVl4TVJHNnUzSkp2THYzYVBNZDExdlNYVm5ZZEZwd0ZiVHNNSFNGZFdqbTQvSE9yRXNSSlBvMmExWTRLQTJYelJqT2p3VWRoRGJyRmdONGFpaUx5WHpaOHd6WUE5aHRpUXlqbzFNeVhlSjd1WG1jWC9leWtPRkV5dU5oQzVidUFqd2JLZkZLWldLVU5qWkx2cWNkWGpQV3Z6THN5dGVvaUh1WHpUOCtPY2luZ1pyQm95bXViWXcrbWQ5YTRiYkw0NDd3MEU3bFJuRWsxZCtpNHdnQ3VKMDZlb21JWDZHci9hSmFqUzZ6bVJldz0iLCJtYWMiOiJkNjIxMjliZjg1NWYzNTZhNDJlYjE0NzNhYTQxN2M0YTI4Y2Q0ZDY2YjJhNzQ0MjNiY2ZiZTI2Y2EzZGQ0YWVkIiwidGFnIjoiIn0%3D; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6ImZIZG1lMU5xQTNqcW9Ja0VLTVBEcUE9PSIsInZhbHVlIjoibFl0VDJKWlNWZXk0TUlOcGdwb3FvZGFKVEczUUdndFhvNy9ldDFFb241T01uRERvdXlTZ3lxZnBpK2hRS04zNjRlM3A5ZHcwMXZHWndUOVpnY2FNdG14OU1LaXFpMFowc2hDQW9KbWp4ODhHdENQK2tuYXhZVlRnSDByWVhlSnZxdFlVeU9QL1YvenNrNUttcWFRVjRsczVnRVJwQzh5SUdHQXM1bURIRkhOanY1RDRSTHp0WGJIY0pxUTNoelczM0pPOVdDUlNhZVB1eVZMbXBXSjh4Vjd0WERBazI2T0RSQm05QTk3ei8xcz0iLCJtYWMiOiJjOWYyNDZkN2Y0NDc4MDJhMTMxYWMxYTY2M2FiOTM2NjE3ODA0MzNhYmEwYzVjYzQ4NDE1NTAwMTkyOTQ2OWY0IiwidGFnIjoiIn0%3D; cstype=list; Hm_ct_3b995bf0c6a621a743d0cf009eaf5c8a=17*1*%E8%B0%B7%E6%AD%8C!2444*1*35965!2445*1*CORICwrfDh8OMw5nDlsOPw4nDkMKEwr3DicOJw43Dkg%3D%3DTOP; HMACCOUNT=78BC3E462729494F; Hm_lvt_3b995bf0c6a621a743d0cf009eaf5c8a=1750391675; _gid=GA1.2.1515404107.1751086118; _gat=1; Hm_lpvt_3b995bf0c6a621a743d0cf009eaf5c8a=1751127082; _ga_856GGDGVGH=GS2.2.s1751125073$o771$g1$t1751127082$j52$l0$h0; XSRF-TOKEN=eyJpdiI6ImFweUVVdkk3b2NtZW9pRGFFQ21KbVE9PSIsInZhbHVlIjoiM1hrUUhGdmZuV3NvYlNFTXBCSy9BWW1NdFR5N2xzQXp1SXQ2anhjeWlDaTFUQUZZUXlRZk1qSWFLdU9UYkExTkZqKys5ZWpZT2hIQ0VpdzFrcEwvYSsrNHJsd0hLTVUxRmZMNnhwRFUxR1o1Z01iY3N4dnpJUG5TR3IwZUplOGsiLCJtYWMiOiI3MDRmMjk3N2IwNWQ1ZGU5NjFjMDg0ZjA4MjczNzFkMzU1ZDg2MTljNjAwM2RhYmY5MmRhNGMxN2NiMTU0YmI5IiwidGFnIjoiIn0%3D; live_tennis_session=eyJpdiI6IlNBQjRrVk40WmNpc3NoYW5Xc2RjSFE9PSIsInZhbHVlIjoiQWY2enhEbUZHemZLUmFod1A1M1RZQXQ2aExUY1lGay91Nzg2U2k5dUxoMjFmeTBxU3ZJTWRISDA4RWpmYXRRVkdTdGtaVnFWZWUyM2gxWm5BczVjNTQzSTFuZEVybi9xaHNRYy81VnJGNWJ5TlR6bUh6WWNiNTBRcTdTS0J5TEsiLCJtYWMiOiIyMWM4M2Q4Y2Q3NmM2NGU0NjZlOTAwOTAxZGI0OTY0NGM4MzMyY2I5NjExM2Q3NzY4MjA1OTNmZjBhYzY4NDhiIiwidGFnIjoiIn0%3D; _ga_7GW8TTD6GW=GS2.1.s1751125060$o818$g1$t1751127125$j57$l0$h0; _ga=GA1.1.985475189.1708225794'

ranker = Ranking(date = date, day = day, wta_dict = wta_dict, atp_dict = atp_dict, 
                 wta_this_week_dict = wta_this_week_dict, wta_url = wta_url, wta_referer = wta_referer, wta_score_cookie = wta_score_cookie,
                 wta_detail_cookie = wta_detail_cookie,
                atp_this_week_dict = atp_this_week_dict, atp_url = atp_url, atp_referer = atp_referer, atp_score_cookie = atp_score_cookie,
                atp_detail_cookie = atp_detail_cookie)

# ranker = Ranking(wta_dict = wta_dict, atp_dict = atp_dict)

In [1925]:
repo_path = '/Users/yinzechen/survivor_champion_ranking/'  # 你的本地仓库路径
commit_message = '自动更新幸存者冠军排名'

In [1977]:
retry_delay = 1  # 失败后等待5秒再重试
while True:
    try:
        ranker.this_week_fetch('wta')
        print('本周数据抓取执行成功')
        break
    except Exception as e:  # 捕获所有异常
        print(f"本周数据抓取执行出错: {str(e)}，{retry_delay}秒后重试...")
        time.sleep(retry_delay)

while True:
    try:
        ranker.fetch('wta')
        print('上周数据抓取执行成功')
        break
    except Exception as e:  # 捕获所有异常
        print(f"上周数据抓取执行出错: {str(e)}，{retry_delay}秒后重试...")
        time.sleep(retry_delay)
while True:
    try:
        ranker.fetch_choice('wta')
        print('本周明细抓取执行成功')
        break
    except Exception as e:  # 捕获所有异常
        print(f"本周明细执行出错: {str(e)}，{retry_delay}秒后重试...")
        time.sleep(retry_delay)
ranker.data_processing('wta')

本周数据抓取执行成功
上周数据抓取执行成功
本周明细执行出错: Expecting value: line 1 column 1 (char 0)，1秒后重试...
本周明细抓取执行成功


/var/folders/5v/b1lp9vjj4877ncw5c4kxj4k80000gn/T/ipykernel_76950/1575374567.py:531: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_week_detail_df['用户名'] = this_week_detail_df['用户名'].apply(lambda x: x[x.rfind('>')+1:])


In [1978]:
# ranker.data_processing('wta')

In [1979]:
# import sys
# del sys.modules['utils']
# from utils import excel_to_html, push_to_github

In [1980]:
output_html = "wta_ranking.html"  # 输出的HTML文件名
write_df = ranker.wta_champion_df.drop('主键', axis=1)
excel_to_html('wta', write_df, repo_path, output_html)

成功生成HTML文件: wta_ranking.html


In [1981]:
push_to_github(output_html, commit_message, repo_path)

🔄 本地有未提交变更，自动 stash...
Saved working directory and index state WIP on main: 25fd40b 自动更新幸存者冠军排名
🔁 还原之前的变更...
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   wta_ranking.html

no changes added to commit (use "git add" and/or "git commit -a")
Dropped refs/stash@{0} (0a5920f3d948bfb02a8a153b2e7071205df905e5)
📦 添加文件到暂存区...
✅ 提交变更...
[main 90d289a] 自动更新幸存者冠军排名
 1 file changed, 2454 insertions(+), 2415 deletions(-)
🚀 推送到 GitHub...
🎉 成功推送到 GitHub！


To github.com:Zechen-3IS/survivor_champion_ranking.git
   25fd40b..90d289a  main -> main


In [1982]:
retry_delay = 1  # 失败后等待5秒再重试
while True:
    try:
        ranker.this_week_fetch('atp')
        print('本周数据抓取执行成功')
        break
    except Exception as e:  # 捕获所有异常
        print(f"本周数据抓取执行出错: {str(e)}，{retry_delay}秒后重试...")
        time.sleep(retry_delay)

while True:
    try:
        ranker.fetch('atp')
        print('上周数据抓取执行成功')
        break
    except Exception as e:  # 捕获所有异常
        print(f"上周数据抓取执行出错: {str(e)}，{retry_delay}秒后重试...")
        time.sleep(retry_delay)
while True:
    try:
        ranker.fetch_choice('atp')
        print('本周明细抓取执行成功')
        break
    except Exception as e:  # 捕获所有异常
        print(f"本周明细执行出错: {str(e)}，{retry_delay}秒后重试...")
        time.sleep(retry_delay)
ranker.data_processing('atp')

本周数据抓取执行成功
上周数据抓取执行成功
本周明细执行出错: Expecting value: line 1 column 1 (char 0)，1秒后重试...
本周明细抓取执行成功


/var/folders/5v/b1lp9vjj4877ncw5c4kxj4k80000gn/T/ipykernel_76950/1575374567.py:531: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_week_detail_df['用户名'] = this_week_detail_df['用户名'].apply(lambda x: x[x.rfind('>')+1:])


In [1983]:
output_html = "atp_ranking.html"  # 输出的HTML文件名
write_df = ranker.atp_champion_df.drop('主键', axis=1)
excel_to_html('atp', write_df,repo_path, output_html)

成功生成HTML文件: atp_ranking.html


In [1984]:
push_to_github(output_html, commit_message, repo_path)

🔄 本地有未提交变更，自动 stash...
Saved working directory and index state WIP on main: 90d289a 自动更新幸存者冠军排名
🔁 还原之前的变更...
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   atp_ranking.html

no changes added to commit (use "git add" and/or "git commit -a")
Dropped refs/stash@{0} (ad8ff7e101abdf1868c799a8f404deb1d7d00898)
📦 添加文件到暂存区...
✅ 提交变更...
[main 88a90c7] 自动更新幸存者冠军排名
 1 file changed, 2332 insertions(+), 2332 deletions(-)
🚀 推送到 GitHub...
🎉 成功推送到 GitHub！


To github.com:Zechen-3IS/survivor_champion_ranking.git
   90d289a..88a90c7  main -> main
